In [75]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [76]:
data=pd.read_csv("D:/my_work/python_codes/git_projects/car_insurance_claim_prediction/dataset/train.csv")
test_x=pd.read_csv("D:/my_work/python_codes/git_projects/car_insurance_claim_prediction/dataset/test.csv")
test_y=pd.read_csv("D:/my_work/python_codes/git_projects/car_insurance_claim_prediction/dataset/sample_submission.csv")

In [77]:
test=pd.concat([test_x,test_y],axis=1)

In [78]:
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [79]:
data.drop("policy_id",axis=1,inplace=True)
test.drop("policy_id",axis=1,inplace=True)

In [80]:
data_x=data.drop("is_claim",axis=1)
data_y=data["is_claim"]

test_x=test.drop("is_claim",axis=1)
test_y=test["is_claim"]

In [81]:
scaler=MinMaxScaler(copy=False)
num_type=data_x.select_dtypes(include="number").columns

data_x[num_type]=scaler.fit_transform(data_x[num_type])
test_x[num_type]=scaler.transform(test_x[num_type])

In [82]:
encoder=OrdinalEncoder()
cat_type=data_x.select_dtypes(include="object").columns

data_x[cat_type]=encoder.fit_transform(data_x[cat_type])
test_x[cat_type]=encoder.transform(test_x[cat_type])


In [83]:
import imblearn
over_samp=imblearn.over_sampling.SMOTE(random_state=0)
data_x,data_y=over_samp.fit_resample(data_x,data_y)

In [84]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_test=train_test_split(data_x,data_y,test_size=0.2)

In [85]:
from sklearn.model_selection import KFold,cross_val_score
from xgboost import XGBClassifier

classifier=XGBClassifier()
k_folds=KFold(n_splits=10)
scores=cross_val_score(classifier,data_x,data_y,cv=k_folds)
scores

array([0.93445164, 0.93262832, 0.93363114, 0.93408697, 0.93299298,
       0.96882122, 0.98222263, 0.98194913, 0.98431802, 0.98340627])

In [86]:
classifier.fit(x_train,y_train)
predict=classifier.predict(x_val)
from sklearn.metrics import f1_score
f1_score(y_test,predict)

0.9573810415886099

In [87]:
new_predict=classifier.predict(test_x)
f1_score(test_y,new_predict)

0.007243973982910343

In [89]:
test_x

,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,max_torque,...,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating
0,0.243199,0.00,0.418919,15.0,0.051764,0.00,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.218455,0.13,0.216216,20.0,0.116270,0.00,2.0,7.0,2.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.4
2,0.233293,0.12,0.229730,20.0,0.116270,0.25,0.0,4.0,2.0,8.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.4
3,0.559521,0.01,0.243243,17.0,0.470987,0.00,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.882893,0.02,0.486486,17.0,0.470987,0.00,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39058,0.032461,0.15,0.635135,11.0,0.365231,0.00,2.0,7.0,2.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.4
39059,0.876747,0.05,0.310811,20.0,0.116270,0.50,4.0,5.0,1.0,4.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.6
39060,0.371296,0.12,0.094595,18.0,0.174474,0.00,2.0,8.0,2.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
39061,0.839121,0.08,0.500000,17.0,0.470987,0.00,3.0,3.0,2.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.4


In [91]:
from sklearn.ensemble import RandomForestClassifier

ra=RandomForestClassifier()
ra.fit(x_train,y_train)
predict=ra.predict(x_val)
from sklearn.metrics import f1_score
print(f1_score(y_test,predict))
new_predict=ra.predict(test_x)
print(f1_score(test_y,new_predict))

0.9113413157540697
0.17692630792924816
